<a href="https://colab.research.google.com/github/fezilemahlangu/My-Research-Project-2022/blob/main/AI_SL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running CNN models using data generated by PPO2


---



---



## Installations 

In [1]:
!pip install zipfile_deflate64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.4 MB/s 


## Imports

In [2]:
import zipfile_deflate64 as zipfile 
import numpy as np
import pandas as pd

from io import BytesIO

import PIL.Image



import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split

import time 
import csv 

from PIL import Image, ImageOps

## Helper functions

In [3]:
def create_train_x(zipfiles,n_s):
    '''
    Function that opens a zipfile of images and returns an array of images. 
    The function also reduces the resolution of the images 

    @Args:
        zfile: zipfile that contains images 

        n: number of images in the zipfile 

        deleted_rows: the images whose information have been deleted through cleaning 

    @returns:
        training_X: array of images in the zip file 
    '''

    training_X=[] 

    for i in range(len(zipfiles)):

      archive = zipfile.ZipFile(zipfiles[i],'r')
      n= n_s[i]

      for i in range(n): #scan through zip file 
        
        filename= BytesIO(archive.read(archive.namelist()[i])) #access an image in the zip file 
        image = PIL.Image.open(filename) #open colour image

        #-------reducing the size/resolution of the image------#
        # width = int(image.size[0] * scale_percent / 100)
        # height = int(image.size[1] * scale_percent / 100)
        width = 84
        height = 84
        dim = (width, height)
        image = ImageOps.grayscale(image) #grayscale 
        # image = image.resize(dim,PIL.Image.ANTIALIAS)

        #-----converting image to array and appending to training_X-----#
        image=np.array(image) #convert image to array
        image = image[:,:,None] #(84,84,1) 
        training_X.append(image/255.0) #-> normalized 

    return training_X


In [4]:
def create_train_y(df_s):
    '''
    This function returns the action column/attribute from the dataframe
    These are the y values 

    @Args:
        df: dataframe containing trial information 
    @returns:
        an array of y values. Actions values taken with image 
    '''
    actions = []
    for df in df_s:
      array = df.to_records(index=False)
      for i in array:
        actions.append(i[0])
      # actions.append(i for i in array['1'])
    
    return actions

    # array = df.to_records(index=False)
    # return array['1']

In [5]:
class timing_Callback(keras.callbacks.Callback):
    '''
    This class takes in the callback of the model and saves the times 
    taken to run each epoch during training in the logs array 
    '''
    def __init__(self):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = time.time()
    def on_epoch_end(self,epoch,logs={}):
        self.logs.append(time.time()-self.starttime)

In [6]:
def run_model_config(train_x,train_y,val_x,val_y,test_x,test_y,img_shape,num_classes,first_layer, sec_layer,third_layer):
    '''
    This function takes in the hyperparameters of the model, builds and runs it.
    It also saves the model hyperparameters and results from training and test scores 
    
    @Args: 
        train_x: x values of the training data set 
        train_y: y values of the training data set 
        val_x: x values of the validation data set 
        val_y: y values of the validation data set 
        test_x: x values of the testing data set 
        test_y: y values of the testing data set 
        img_shape: the shape of the images; 3 by 3
        num_classes: number of actions a player can make 
        first_layer: the hyperparameters of the first layer of the CNN
        second_layer: the hyperparameters of the second layer of the CNN
        fieldnames_results: field names of results.csv
        fieldnames_models: field names of models.csv


    @returns: Nothing 
    '''
    #building model 
    model = keras.Sequential()
    model.add(Conv2D(filters=first_layer[0],kernel_size=first_layer[1],strides=first_layer[2], padding=first_layer[3],activation=first_layer[4],input_shape=img_shape))
    model.add(MaxPooling2D(pool_size=first_layer[5]))
    model.add(Dropout(rate=first_layer[6]))
    if(len(sec_layer)>1):
        model.add(Conv2D(filters=sec_layer[0],kernel_size=sec_layer[1],strides=sec_layer[2],padding=sec_layer[3],activation=sec_layer[4]))
        model.add(MaxPooling2D(pool_size=sec_layer[5]))
        model.add(Dropout(rate=sec_layer[6]))
    
    if(len(third_layer)>1):
        model.add(Conv2D(filters=third_layer[0],kernel_size=third_layer[1],strides=third_layer[2],padding=third_layer[3],activation=third_layer[4]))
        model.add(MaxPooling2D(pool_size=third_layer[5]))
        model.add(Dropout(rate=third_layer[6]))

    model.add(Flatten())
    model.add(Dense(units=first_layer[7], activation = 'relu'))
    model.add(Dropout(rate=first_layer[8]))
    model.add(Dense(num_classes, activation = 'softmax'))

    #compile model
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=first_layer[9]),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

    model.summary()

    #early stopping 
    early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0.00001, patience = 3)

    #run model 
    time_callback = timing_Callback() 
    start = time.time()
    Atari_Conf_History =  model.fit(train_x, train_y, epochs = 25, validation_data=(val_x, val_y), callbacks=[time_callback,early_stopping])
    total_time = time.time()-start 

    #save model hyperparameters to csv
    # save_model(fieldnames_model,first_layer,sec_layer,third_layer)

    #save results to csv
    # history = Atari_Conf_History.history 
    test_loss, test_acc = model.evaluate(test_x, test_y)
    # save_results(fieldnames_results,total_time,time_callback,test_acc,test_loss)

    return  

In [7]:
def get_data(trial):

    game_folder= str("/content/drive/MyDrive/AI_atari_data/enduro/")

    zipname=game_folder+"trial"+str(trial)+".zip"
    # textfile=game_folder+str(trial)+".txt"
    csv=game_folder+"trial"+str(trial)+"_actions.csv"

    # read_file=pd.read_csv(textfile)
    # read_file.to_csv(csv,index=True)
    df= pd.read_csv(csv)

    return zipname, df 

## Get data 

In [8]:
zipfiles=[]
n_s=[]
df_s=[]

#-----trial = 1------#
trial = 1
zipname,df= get_data(trial) #get zipname and csv in form of dataframe 
n=len(df) #number of frames in trial
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)


#-----trial = 2------#
trial = 2 #trial 2
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)

#-----trial = 3------#
trial = 3 
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)

#-----trial = 4------#
trial = 4
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)

#-----trial = 5------#
trial = 5
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)


#-----trial = 6------#
trial = 6
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)

#-----trial = 7------#
trial = 7
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)

#-----trial = 8------#
trial = 8
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)

#-----trial = 9------#
trial = 9
zipname,df= get_data(trial)
n=len(df)
zipfiles.append(zipname)
n_s.append(n)
df_s.append(df)



In [9]:
n_s

[6648, 3325, 6649, 6647, 3321, 3319, 6650, 6649, 3325]

In [10]:
#----------------------------------#
images = create_train_x(zipfiles,n_s)
actions = create_train_y(df_s)


In [11]:
try:
  len(images) == np.sum(n_s) and len(actions) == np.sum(n_s)
except:
  print("oops something is wrong, sizes do not match")
  

## Run 

In [12]:


import csv 

#-----------------------------------------------------------------#
#prepare csv for results 
# fieldnames_results = ['total_time', 'call_back_time','test_acc','test_loss'] 
# with open('/home-mscluster/fmahlangu/2089676/atari_enduro_data/results.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.DictWriter(f, fieldnames=fieldnames_results)
#     writer.writeheader()

# #prepare csv for results 
# fieldnames_m = ['first_layer','second_layer','third_layer'] 
# with open('/home-mscluster/fmahlangu/2089676/atari_enduro_data/models.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.DictWriter(f, fieldnames=fieldnames_m)
#     writer.writeheader()

#-----------retrieving training data----------------------------#


# metaData_df= pd.read_csv('/home-mscluster/fmahlangu/2089676/atari_enduro_data/meta_data.csv')

# dataframe= metaData_df[metaData_df.GameName=="enduro"]





# images = create_train_x(zipname,n)
# n=dataframe[dataframe.trial_id==trial].total_frame.tolist() #get number of frame/images
# n=n[0]

# df,deleted_rows=clean_df(df)
# images=create_train_x(zipname,n,deleted_rows)
# actions=create_train_y(df)

trainX, valX, trainY, valY = train_test_split(images, actions, test_size=0.18)
trainX, testX, trainY, testY = train_test_split(trainX, trainY, test_size=0.12)

# #-----------#
trainX = np.array(trainX)
valX = np.array(valX)
testX = np.array(testX)

trainY = np.array(trainY)
valY = np.array(valY)
testY = np.array(testY)







# #---------converting y values to categorical data (one-hot encoding)---#

img_shape = [84, 84, 1]
num_classes = 18

trainY = keras.utils.to_categorical(trainY, num_classes)
valY = keras.utils.to_categorical(valY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)

## Running CNN models 

In [13]:
params = []
    
params.append([[16, 1, 1, 'same', 'relu', 2, 0.05, 32, 0.05, 0.001],[],[]])
# params.append([[16, 5, 1, 'same', 'relu', 2, 0.05, 64, 0.1, 0.01],[],[]])
# params.append([[16, 3, 3, 'same', 'relu', 1, 0.05, 128, 0.05, 0.02],[64, 2, 2, 'same', 'relu', 3, 0.15],[512, 2, 1, 'same', 'relu', 2, 0.15]])
# params.append([[16, 7, 3, 'same', 'relu', 2, 0.05, 128, 0.05, 0.02],[32, 4, 2, 'same', 'relu', 3, 0.15],[52, 5, 1, 'same', 'relu', 2, 0.15]])
# params.append([[32, 4, 1, 'same', 'relu', 2, 0.03, 64, 0.03, 0.004],[64, 3, 3, 'same', 'relu', 2, 0.15],[512, 7, 1, 'same', 'relu', 3, 0.05]])
# params.append([[32, 3, 2, 'same', 'relu', 3, 0.25, 128, 0.25, 0.00005],[64, 2, 2, 'same', 'relu', 3, 0.15],[256, 5, 1, 'same', 'relu', 2, 0.35]])
params.append([[18, 2, 1, 'same', 'relu', 2, 0.01, 512, 0.15, 0.000003],[50, 1, 1, 'same', 'relu', 2, 0.15],[128, 3, 1, 'same', 'relu', 3, 0.2]])
# params.append([[16, 7, 1, 'same', 'relu', 2, 0.05, 128, 0.05, 0.02],[],[]])
# params.append([[16, 8, 1, 'same', 'relu', 2, 0.05, 128, 0.1, 0.0001],[],[]])
# params.append([[16, 3, 1, 'same', 'relu', 2, 0.05, 128, 0.25, 0.00001],[],[]])
# params.append([[16, 5, 1, 'same', 'relu', 2, 0.05, 256, 0.15, 0.04],[],[]])
# params.append([[16, 7, 1, 'same', 'relu', 2, 0.1, 256, 0.05, 0.01],[],[]])
# params.append([[16, 5, 1, 'same', 'relu', 2, 0.15, 256, 0.05, 0.01],[],[]])
# params.append([[16, 8, 1, 'same', 'relu', 2, 0.25, 256, 0.15, 0.0003],[],[]])
# params.append([[32, 5, 1, 'same', 'relu', 2, 0.05, 128, 0.05, 0.0005],[],[]])
# params.append([[32, 3, 1, 'same', 'relu', 2, 0.05, 256, 0.05, 0.001],[],[]])
# params.append([[64, 8, 1, 'same', 'relu', 2, 0.05, 512, 0.05, 0.0001],[],[]])
# params.append([[64, 5, 1, 'same', 'relu', 2, 0.05, 128, 0.05, 0.001],[],[]])
# params.append([[128, 3, 1, 'same', 'relu', 2, 0.05, 128, 0.05, 0.0001],[],[]])
# params.append([[256, 5, 1, 'same', 'relu', 2, 0.05, 128, 0.05, 0.0001],[],[]])
# params.append([[32, 3, 1, 'same', 'relu', 2, 0.05, 256, 0.05, 0.0001],[16, 1, 1, 'same', 'relu', 2, 0.15],[]])
# params.append([[32, 3, 1, 'same', 'relu', 2, 0.05, 256, 0.05, 0.0001],[32, 3, 1, 'same', 'relu', 3, 0.25],[]])
# params.append([[16, 5, 1, 'same', 'relu', 2, 0.05, 128, 0.05, 0.01],[16, 7, 2, 'valid', 'relu', 2, 0.25],[]])
# params.append([[16, 7, 1, 'same', 'relu', 2, 0.05, 256, 0.05, 0.003],[32, 3, 1, 'same', 'relu', 3, 0.25],[]])
# params.append([[32, 3, 2, 'same', 'relu', 2, 0.05, 128, 0.05, 0.000001],[128, 3, 2, 'same', 'relu', 3, 0.25],[]])
# params.append([[16, 8, 1, 'same', 'relu', 2, 0.05, 256, 0.05, 1],[32, 3, 1, 'same', 'relu', 3, 0.25],[]])
# params.append([[32, 3, 2, 'same', 'relu', 2, 0.05, 128, 0.05, 0.000000000000001],[128, 3, 2, 'same', 'relu', 3, 0.25],[]])
# params.append([[32, 3, 2, 'same', 'relu', 2, 0.05, 512, 0.05, 0.0001],[128, 3, 2, 'same', 'relu', 3, 0.25],[]])
# params.append([[16, 5, 2, 'same', 'relu', 2, 0.05, 128, 0.05, 0.01],[32, 3, 1, 'same', 'relu', 3, 0.25],[32, 3, 1, 'same', 'relu', 3, 0.25]])
# params.append([[16, 8, 2, 'same', 'relu', 2, 0.05, 128, 0.05, 0.01],[32, 3, 2, 'same', 'relu', 3, 0.25],[64, 3, 1, 'same', 'relu', 3, 0.25]])
# params.append([[32, 5, 2, 'same', 'relu', 2, 0.05, 512, 0.05, 0.00001],[64, 3, 2, 'same', 'relu', 3, 0.25],[64, 3, 1, 'same', 'relu', 3, 0.25]])
# params.append([[16, 7, 4, 'same', 'relu', 1, 0.05, 256, 0.05, 0.001],[32, 4, 2, 'same', 'relu', 1, 0.25],[64, 3, 1, 'same', 'relu', 1, 0.25]]) #minh
# params.append([[32, 8, 4, 'same', 'relu', 1, 0.05, 256, 0.05, 0.1],[64, 4, 2, 'same', 'relu', 1, 0.25],[128, 3, 1, 'same', 'relu', 1, 0.25]])
# params.append([[32, 8, 4, 'same', 'relu', 1, 0.05, 256, 0.05, 0.1],[64, 4, 2, 'same', 'relu', 1, 0.25],[128, 3, 1, 'same', 'relu', 1, 0.25]])


for p in params:
  first = p[0]
  second = p[1]
  third = p[2]

  run_model_config(trainX,trainY,valX,valY,testX,testY,img_shape,num_classes,first,second,third)

  # run_model_config(trainX,trainY,valX,valY,testX,testY,img_shape,num_classes,first,second,third,fieldnames_results,fieldnames_m)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 84, 84, 16)        32        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 42, 42, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 42, 42, 16)        0         
                                                                 
 flatten (Flatten)           (None, 28224)             0         
                                                                 
 dense (Dense)               (None, 32)                903200    
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                        

KeyboardInterrupt: ignored